In [5]:
# 캘리 파일 테스트

In [6]:
# 캘리 파일 테스트
import numpy as np
import cv2
import os

In [7]:
CALIB_FILE = "stereo_calib.npz"
CAMERA_INDEX = 0

# 해상도
FRAME_WIDTH = 1280
FRAME_HEIGHT = 480

# 캘리 데이터
if not os.path.exists(CALIB_FILE):
    print(f"에러: '{CALIB_FILE}' 파일이 없습니다.")
    exit()

print("캘리 데이터 로드")
data = np.load(CALIB_FILE)
mtxL, distL = data['mtxL'], data['distL']
mtxR, distR = data['mtxR'], data['distR']
R, T = data['R'], data['T']

# 카메라
cap = cv2.VideoCapture(CAMERA_INDEX)
if not cap.isOpened():
    print("카메라 연결 실패")
    exit()

cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*'MJPG'))
cap.set(cv2.CAP_PROP_FRAME_WIDTH, FRAME_WIDTH)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, FRAME_HEIGHT)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
half_width = width // 2

if width == 0:
    print("카메라 해상도 가져오기 실패")
    cap.release()
    exit()

print(f"카메라 해상도: {width}x{height}")

캘리 데이터 로드
카메라 해상도: 1280x480


In [8]:
# Rectification 변환 행렬
R1, R2, P1, P2, Q, roi1, roi2 = cv2.stereoRectify(
    mtxL, distL, mtxR, distR, (half_width, height), R, T, alpha=0
)

# 매핑 함수
map1_L, map2_L = cv2.initUndistortRectifyMap(mtxL, distL, R1, P1, (half_width, height), cv2.CV_16SC2)
map1_R, map2_R = cv2.initUndistortRectifyMap(mtxR, distR, R2, P2, (half_width, height), cv2.CV_16SC2)

# print("화면의 '녹색/빨간색 가로선'이 양쪽에서 일치하는지 확인하세요.")
# print("'q'를 누르면 종료됩니다.")

# 화면 출력
while True:
    ret, frame = cap.read()
    if not ret:
        break

    imgL_raw, imgR_raw = frame[:, :half_width], frame[:, half_width:]
    imgL_rect = cv2.remap(imgL_raw, map1_L, map2_L, cv2.INTER_LINEAR)
    imgR_rect = cv2.remap(imgR_raw, map1_R, map2_R, cv2.INTER_LINEAR)

    res_raw = np.hstack((imgL_raw, imgR_raw))
    res_rect = np.hstack((imgL_rect, imgR_rect))

    for line_y in range(0, res_rect.shape[0], 30):
        cv2.line(res_raw, (0, line_y), (res_raw.shape[1], line_y), (0, 0, 255), 1)
        cv2.line(res_rect, (0, line_y), (res_rect.shape[1], line_y), (0, 255, 0), 1)

    combined_view = np.vstack((res_raw, res_rect))
    cv2.imshow("Stereo Calibration Test (Top: Raw, Bottom: Rectified)", combined_view)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()